In [81]:
# notebook parameters
id = "dimMovies-Gold"


In [82]:
from common_function import readConfig, absPath, updatePipelineStatus,createHashKey
import pandas as pd
import os


In [83]:
config = readConfig(id)
config

✓ Config loaded successfully: dimMovies-Gold.json


{'pipeline_id': 'dimMovies-Gold',
 'dependency': 'OMDB-Silver',
 'target': {'pk': ['_sk_movie'],
  'format': 'parquet',
  'name': 'dimMovies',
  'path': 'data\\03_gold\\',
  'mode': 'overwrite'}}

In [84]:
# Read data

path = os.path.join(absPath(), "data\\02_silver\\omdb")

df_omdb = pd.read_parquet(path, engine='fastparquet')

In [85]:
# Read data

path = os.path.join(absPath(), "data\\02_silver\\revenues")
df_reve = pd.read_parquet(path, engine='fastparquet')

In [86]:
# create list of movies from revenue (master table) and enrich them with available data from omdb
df_reve = df_reve[['title']]

df_reve = createHashKey(
    df=df_reve,
    key_columns=['title'],
    hash_column='_sk_movie'
)

#dropping duplicates
df_reve = df_reve[["_sk_movie","title"]].drop_duplicates()

# create hash for omdb
df_omdb = createHashKey(
    df=df_omdb,
    key_columns=['title'],
    hash_column='_sk_movie'
)


df = df_reve.merge(
    df_omdb,
    on='_sk_movie',
    how='left'

)

#adding a flag for enriched data
df['is_enriched'] = df['title_y'].notna().astype(int) 

#renaming columns
df = df.rename(columns={
    'title_x': 'title'}
)

In [87]:
df["imdb_rating"] = pd.to_numeric(
    df["imdb_rating"].str.split("/").str[0], 
    errors='coerce'
)

In [88]:
df.columns

Index(['_sk_movie', 'title', 'title_y', 'year', 'rated', 'released', 'runtime',
       'genre', 'director', 'writer', 'actors', 'plot', 'language', 'country',
       'awards', 'poster', 'imdb_rating', 'rotten_tomatoes', 'metacritic',
       'metascore', 'imdb_votes', 'imdb_id', 'box_office', 'production',
       'website', '_tf_ingestion_date', '_tf_ingestion_time', 'is_enriched'],
      dtype='object')

In [89]:
df = df[['_sk_movie', 'title', 'year', 'rated', 'released', 'runtime',
       'genre', 'director', 'writer', 'actors', 'plot', 'language', 'country',
       'awards', 'poster', 'imdb_rating', 'rotten_tomatoes', 'metacritic',
       'metascore', 'imdb_votes', 'imdb_id', 'box_office', 'production',
       'is_enriched']]

In [90]:
target_path = os.path.join(absPath(), config["target"]["path"],config["target"]["name"])

df.to_parquet(target_path, index=False)

In [91]:
updatePipelineStatus(id)

✓ Updated pipeline status: dimMovies-Gold - success
